**Group Assignment**

**T04 Group 45**

**Facial Expression Recognition**


---



Facial Expression Recognition (FER) is a crucial task in computer vision, with applications ranging from emotion analysis to human-computer interaction. This project explores the effectiveness of different neural network architectures (CNN, VGG16, ResNet) and baseline models (KNN, SVM, Random Forest) for recognizing facial expressions. The dataset used for this project is taken from FER-2013, it consists of grayscale images of faces, each labeled with one of several emotions.

In [ ]:
# Mount Google Drive to access the dataset
from google.colab import drive
drive.mount('/content/drive')

# Load and Preprocessing the data

The dataset is stored in a google drive directory where each subdirectory represents a class label (e.g., "happy," "sad"). The images are loaded using OpenCV in grayscale mode, resized to 48x48 pixels, and normalized to have pixel values between 0 and 1. This preprocessing step ensures that the images are uniform in size and the model training process is stable and efficient.

The labels, originally in string format (e.g., "happy," "sad"), are converted to numerical values using label encoding, followed by one-hot encoding to make them suitable for multi-class classification. The dataset is then split into training, validation, and test sets, ensuring that the models can be evaluated on unseen data.

In [ ]:
import numpy as np
import cv2
import os
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# Define the path to the dataset
data_dir = '/content/drive/MyDrive/archive/Fer2013_train'

# Initialize lists to store images and their corresponding labels
images = []
labels = []

# Load and preprocess images by
# Looping through each label directory in the dataset
for label in os.listdir(data_dir):
    class_dir = os.path.join(data_dir, label)
    if os.path.isdir(class_dir): # Ensure it's a directory
        for img_file in os.listdir(class_dir):
            img_path = os.path.join(class_dir, img_file)
            img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE) # Load the image in grayscale mode
            img = cv2.resize(img, (48, 48))  # Resize to the desired size (48x48)
            img = img / 255.0  # Normalize the image to [0, 1]
            images.append(img) # Append the processed image to the images list
            labels.append(label) # Append the label to the labels list

# Convert the lists of images and labels into NumPy arrays
X = np.array(images).reshape(-1, 48, 48, 1) # Reshape images to include channel dimension
y = np.array(labels)

# Encode labels to integers
le = LabelEncoder()
y = le.fit_transform(y) # Transform labels to integer encoding
y = to_categorical(y, num_classes=len(np.unique(y)))# One-hot encode the labels

# Splitting into train, validation, and test sets
X_train, X_val_test, y_train, y_val_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_val_test, y_val_test, test_size=0.5, random_state=42)

# Advance Models

# Convolutional Neural Network (CNN)

A CNN architecture is implemented with the following layers:



*   **Convolutional Layers**: Two convolutional layers with ReLU activation functions extract features from the images.
*   **Pooling Layers**: MaxPooling layers reduce the spatial dimensions of the feature maps, helping in down-sampling.
*   **Fully Connected Layer**: A Dense layer with 128 neurons, followed by a Dropout layer to prevent overfitting.
*   **Output Layer**: A softmax activation function is used to predict the probability distribution across the different classes.

The model is compiled using the Adam optimizer and categorical cross-entropy loss, and it is trained for 25 epochs.











In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

# Function to create a CNN model
def create_cnn_model():
    model = Sequential()
     # First convolutional layer with 32 filters, 3x3 kernel, and ReLU activation
    model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(48, 48, 1)))
    model.add(MaxPooling2D(pool_size=(2, 2)))  # Max pooling with 2x2 pool size
    # Second convolutional layer with 64 filters
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2))) # Max pooling with 2x2 pool size
    model.add(Flatten()) # Flatten the output from the convolutional layers
    # Dense layer with 128 neurons and ReLU activation
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5)) # Dropout to prevent overfitting
    # Output layer with softmax activation for multi-class classification
    model.add(Dense(len(np.unique(labels)), activation='softmax'))
    return model

# Create and compile the CNN model
cnn_model = create_cnn_model()
cnn_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
# Train the CNN model
cnn_model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=25, batch_size=64)



# VGG16

A pre-trained deep convolutional network, is fine-tuned for this task. The model is loaded with pre-trained ImageNet weights, and the top classification layers are removed. The following layers are added:



*   **Flatten Layer**: Converts the 3D output of VGG16 to 1D.
*   **Dense Layer**: A Dense layer with 256 neurons and ReLU activation.
*   Output Layer: A softmax activation function for multi-class classification.


To match VGG16's input requirements (3-channel images), the grayscale images are replicated across three channels. The model is then compiled and trained similarly to the CNN model.

In [ ]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Input,Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np

# Function to create a VGG16-based model
def create_vgg16_model():
    # Load the VGG16 model pretrained on ImageNet, excluding the top classification layers
    vgg16 = VGG16(weights='imagenet', include_top=False, input_tensor=Input(shape=(48, 48, 3)))
    model = Sequential()
    model.add(vgg16) # Add the VGG16 model as the base
    model.add(Flatten()) # Flatten the output of VGG16
    # Dense layer with 256 neurons and ReLU activation
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.5)) # Dropout to prevent overfitting
    # Output layer with softmax activation for multi-class classification
    model.add(Dense(len(np.unique(labels)), activation='softmax'))
    return model

# Convert grayscale images to 3 channels to match VGG16 input requirements
X_train_vgg16 = np.repeat(X_train, 3, axis=-1)
X_val_vgg16 = np.repeat(X_val, 3, axis=-1)
X_test_vgg16 = np.repeat(X_test, 3, axis=-1)

# Create and compile the VGG16 model
vgg16_model = create_vgg16_model()
vgg16_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
# Train the VGG16 model
vgg16_model.fit(X_train_vgg16, y_train, validation_data=(X_val_vgg16, y_val), epochs=25, batch_size=64)



# ResNet50

 Another pre-trained deep network, is employed to test the performance of a deeper architecture. Similar to VGG16, ResNet50 is loaded with pre-trained ImageNet weights, and additional layers are added for the classification task. The final architecture is similar to VGG16, with a Flatten layer, a Dense layer, and a softmax output layer.

In [ ]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Input,Flatten, Dense, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np

# Function to create a ResNet-based model
def create_resnet_model():
    # Load the ResNet50 model pretrained on ImageNet, excluding the top classification layers
    resnet = ResNet50(weights='imagenet', include_top=False, input_tensor=Input(shape=(48, 48, 3)))
    model = Sequential()
    model.add(resnet) # Add the ResNet model as the base
    model.add(Flatten()) # Flatten the output of ResNet
    # Dense layer with 256 neurons and ReLU activation
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.5)) # Dropout to prevent overfitting
    # Output layer with softmax activation for multi-class classification
    model.add(Dense(len(np.unique(labels)), activation='softmax'))
    return model

# Compile and train the ResNet model
resnet_model = create_resnet_model()
resnet_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
resnet_model.fit(X_train_resnet, y_train, validation_data=(X_val_resnet, y_val), epochs=25, batch_size=64)


# Metrics Calculation

Each model is evaluated on the test set using several metrics:


*   **Accuracy**: The overall correctness of the model's predictions.
*   **Precision, Recall, F1 Score**: These metrics provide insight into the model's performance on each class, particularly in imbalanced datasets.
*   **Mean Squared Error (MSE) & Mean Absolute Error (MAE)**: These metrics measure the average errors in the model’s predictions.
*   **Cross-Entropy Loss**: This loss measures the difference between the true labels and the predicted probabilities.







In [ ]:
from sklearn.metrics import precision_score, accuracy_score, f1_score, mean_squared_error, mean_absolute_error, log_loss
import numpy as np

# Function to calculate and print various metrics
def calculate_metrics(model, X_test, y_test):
    # Get predictions
    y_pred_prob = model.predict(X_test)  # Predicted probabilities
    y_pred = np.argmax(y_pred_prob, axis=1)  # Predicted class labels
    y_true = np.argmax(y_test, axis=1)  # True class labels

    # Calculate metrics
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred, average='weighted')
    f1 = f1_score(y_true, y_pred, average='weighted')
    mse = mean_squared_error(y_true, y_pred)
    mae = mean_absolute_error(y_true, y_pred)
    cross_entropy = log_loss(y_true, y_pred_prob)

    # Print metrics
    print(f'Accuracy: {accuracy:.4f}')
    print(f'Precision: {precision:.4f}')
    print(f'F1 Score: {f1:.4f}')
    print(f'Mean Squared Error (MSE): {mse:.4f}')
    print(f'Mean Absolute Error (MAE): {mae:.4f}')
    print(f'Cross-Entropy Loss: {cross_entropy:.4f}')

# Calculate and print metrics for each model
print("Metrics for CNN Model:")
calculate_metrics(cnn_model, X_test, y_test)

print("Metrics for VGG16 Model:")
calculate_metrics(vgg16_model, X_test_vgg16, y_test)

print("Metrics for ResNet Model:")
calculate_metrics(resnet_model,X_test_resnet, y_test)

# Baseline Models

To provide a comparative analysis, three baseline models are implemented:



*  **K-Nearest Neighbors (KNN)**: This model classifies an image based on the majority class among its k-nearest neighbors in the feature space.
*   **Support Vector Machine (SVM)**: An SVM model is trained to find the optimal hyperplane that separates the classes in the feature space.
*   **Random Forest (RF)**: A Random Forest model, which is an ensemble of decision trees, is trained to classify the images.

The images are flattened into 1D vectors, standardized, and then fed into these baseline models.

In [ ]:
import os
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, roc_auc_score, mean_absolute_error, mean_squared_error
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define image data generator for dataset loading
datagen = ImageDataGenerator(
    rescale=1./255,         # Rescale pixel values to [0, 1]
    validation_split=0.2    # Split data into training and validation sets
)

# Specify the paths to your dataset
dataset_path = '/content/drive/MyDrive/Fer2013_train'

# Create data generators for training and validation sets
train_generator = datagen.flow_from_directory(
    dataset_path,
    target_size=(48, 48),   # Resize images to 48x48
    color_mode='grayscale', # Convert images to grayscale
    batch_size=32,          # Number of images to yield per batch
    class_mode='categorical', # Use categorical labels (for multi-class classification)
    subset='training',      # Set as training data
    shuffle=True
)

validation_generator = datagen.flow_from_directory(
    dataset_path,
    target_size=(48, 48),   # Resize images to 48x48
    color_mode='grayscale', # Convert images to grayscale
    batch_size=32,          # Number of images to yield per batch
    class_mode='categorical', # Use categorical labels (for multi-class classification)
    subset='validation',    # Set as validation data
    shuffle=True
)

# Function to extract data from the generator
def extract_data(generator):
    data = []
    labels = []
    for i in range(len(generator)):
        imgs, lbls = generator[i]
        data.extend(imgs)
        labels.extend(lbls)
    return np.array(data), np.array(labels)

# Extract training and validation data
X_train, y_train = extract_data(train_generator)
X_val, y_val = extract_data(validation_generator)

# Flatten the images for baseline models (KNN, SVM, RF)
X_train_flat = X_train.reshape(X_train.shape[0], -1)
X_val_flat = X_val.reshape(X_val.shape[0], -1)

# Convert one-hot labels to categorical labels
y_train_cat = np.argmax(y_train, axis=1)
y_val_cat = np.argmax(y_val, axis=1)

# Standardize the data using StandardScaler
scaler = StandardScaler()
X_train_flat = scaler.fit_transform(X_train_flat)
X_val_flat = scaler.transform(X_val_flat)

# Baseline Model: K-Nearest Neighbors (KNN)
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train_flat, y_train_cat)
y_pred_knn = knn.predict(X_val_flat)

# Baseline Model: Support Vector Machine (SVM)
svm = SVC(probability=True)
svm.fit(X_train_flat, y_train_cat)
y_pred_svm = svm.predict(X_val_flat)

# Baseline Model: Random Forest (RF)
rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train_flat, y_train_cat)
y_pred_rf = rf.predict(X_val_flat)

# Function to evaluate baseline models
def evaluate_model(y_true, y_pred, model_name):
    accuracy = accuracy_score(y_true, y_pred)
    precision, recall, f1, _ = precision_recall_fscore_support(y_true, y_pred, average='weighted')
    # Calculate ROC AUC only for RF and SVM as KNN does not output probabilities
    roc_auc = roc_auc_score(y_true, rf.predict_proba(X_val_flat), multi_class='ovr') if model_name == 'RF' else roc_auc_score(y_true, svm.predict_proba(X_val_flat), multi_class='ovr') if model_name == 'SVM' else 'N/A'
    mae = mean_absolute_error(y_true, y_pred)
    mse = mean_squared_error(y_true, y_pred)
    # Print evaluation metrics
    print(f"{model_name} Accuracy: {accuracy}")
    print(f"{model_name} Precision: {precision}")
    print(f"{model_name} Recall: {recall}")
    print(f"{model_name} F1 Score: {f1}")
    print(f"{model_name} ROC AUC: {roc_auc}")
    print(f"{model_name} MAE: {mae}")
    print(f"{model_name} MSE: {mse}")

# Evaluate and print metrics for each baseline model
evaluate_model(y_val_cat, y_pred_knn, 'KNN')
evaluate_model(y_val_cat, y_pred_svm, 'SVM')
evaluate_model(y_val_cat, y_pred_rf, 'RF')

# Testing

A testing module is implemented to allow real-time predictions. Users can upload images, and the trained models will predict the emotion displayed in the image. The function preprocesses the image similarly to the training data and uses the trained models to predict the emotion with an associated confidence score.

In [ ]:
import numpy as np
from tensorflow.keras.preprocessing import image
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from google.colab import files

# Function to load and preprocess the image for prediction
def load_and_preprocess_image(img_path, target_size=(48, 48), grayscale=False):
    if grayscale:
        img = image.load_img(img_path, target_size=target_size, color_mode='grayscale')
    else:
        img = image.load_img(img_path, target_size=target_size)

    img_array = image.img_to_array(img) # Convert image to array
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    img_array /= 255.0  # Rescale to [0, 1]
    return img_array

# Function to predict the class of the image
def predict_emotion(img_path, model, model_name):
    class_labels = ['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']
    img_array = load_and_preprocess_image(img_path, grayscale=(model_name == 'CNN'))
    predictions = model.predict(img_array)
    predicted_class = np.argmax(predictions)
    return class_labels[predicted_class], np.max(predictions)

# Upload and predict
uploaded = files.upload()
for img_path in uploaded.keys():
    # Display the image
    img = mpimg.imread(img_path)
    plt.imshow(img)
    plt.axis('off')
    plt.show()

    # Predict the emotion using CNN model
    predicted_emotion, confidence = predict_emotion(img_path, cnn_model, 'CNN')
    print(f'CNN Model Prediction: The predicted emotion for {img_path} is: {predicted_emotion} with confidence {confidence*100:.2f}%')

    # Predict the emotion using VGG16 model
    predicted_emotion, confidence = predict_emotion(img_path, vgg16_model, 'VGG16')
    print(f'VGG16 Model Prediction: The predicted emotion for {img_path} is: {predicted_emotion} with confidence {confidence*100:.2f}%')

    # Predict the emotion using ResNet model
    predicted_emotion, confidence = predict_emotion(img_path, resnet_model, 'ResNet')
    print(f'ResNet Model Prediction: The predicted emotion for {img_path} is: {predicted_emotion} with confidence {confidence*100:.2f}%')


# Conclusion

This project demonstrates the effectiveness of advanced neural network architectures (VGG16 and ResNet50) in facial expression recognition, outperforming simpler CNN models and baseline models like KNN, SVM, and Random Forest. The use of transfer learning with pre-trained models like VGG16 and ResNet50 significantly enhances the accuracy and robustness of the emotion recognition system.

This comparative study underscores the importance of choosing the right model architecture for specific computer vision tasks and highlights the potential of deep learning in understanding and interpreting human emotions.